# Create Tracking Sheet by Type from Master
This notebook updates the individual, type level tracking CSV files from the main tracking CSV.

In [4]:
import numpy as np
import pandas as pd

In [5]:
tracking = pd.read_csv('../tracking/tracking.csv')

In [6]:
#types = ['2JHN', '3JHN', '2JHC', '3JHC', '2JHH', '1JHN', '3JHH', '1JHC']
# types = ['3JHN', '2JHH'] #, '1JHN', '3JHH', '1JHC', '2JHN', '2JHC', '3JHC']
types = ['2JHH']
#types = ['3JHC']
for t in types:
    print(t)
    base_cols = ['run_id',
                 'model_number',
                 'eval_metric',
                 'n_estimators',
                 'model_type',
                 'early_stopping_rounds',
                 'random_state',
                 'n_threads',
                 'learning_rate',
                 'n_fold',
                 'training_time',
                 'n_features',
                 'depth',
                 'gml_oof_score',
                 'oof_score',
                 'lb_score',]
    t_cols = [x for x in tracking.columns if t in x]
    print(t_cols)
    t_filtered = tracking[base_cols + t_cols]
    t_filtered = t_filtered.dropna(subset=[f'{t}_mae_cv']).sort_values(f'{t}_mae_cv')
    sec_cols = [x for x in t_filtered.columns if 'sec' in x]
    for s in sec_cols:
        t_filtered[s.replace('sec','hour')] = (t_filtered[s] / 60 / 60).round(4)
    t_filtered = t_filtered.drop(sec_cols, axis=1)
    t_filtered[f'{t}_logmae_cv'] = t_filtered[f'{t}_mae_cv'].apply(np.log).round(5)
    t_filtered = t_filtered.round(5)
    hour_cols = [x for x in t_filtered if 'hour' in x]
    t_filtered['hours_total'] = t_filtered[hour_cols].sum(axis=1)
    collist = [x for x in t_filtered.columns]
    first_cols = ['run_id','model_number',f'{t}_mae_cv',f'{t}_logmae_cv','depth',
                  'learning_rate','model_type','n_fold','eval_metric','n_features', f'{t}_features',
                  'random_state','hours_total',
                  f'{t}_meta_fc_mae_cv_f1', f'{t}_meta_fc_mae_cv_f2', f'{t}_meta_fc_mae_cv_f3']

    _ = [collist.remove(x) for x in first_cols]
    new_collist = first_cols + collist
    t_filtered = t_filtered[new_collist]
    
    t_filtered['n_estimators'] = t_filtered['n_estimators'].astype(int)
    t_filtered['early_stopping_rounds'] = t_filtered['early_stopping_rounds'].astype(int)
    t_filtered['n_threads'] = t_filtered['n_threads'].astype(int)
    try:
        t_filtered['n_features'] = t_filtered['n_features'].astype(int)
    except:
        pass
    try:
        t_filtered[f'{t}_features'] = t_filtered[f'{t}_features'].astype(int)
    except:
        pass
    t_filtered['random_state'] = t_filtered['random_state'].astype(int)
    t_filtered['n_fold'] = t_filtered['n_fold'].astype(int)

    t_filtered.to_csv(f'../tracking/tracking_{t}.csv', index=False, float_format='%.4f')

2JHH
['2JHH_mae_cv', '2JHH_std_mae_cv', '2JHH_tr_sec_f1', '2JHH_pred_sec_f1', '2JHH_tr_sec_f2', '2JHH_pred_sec_f2', '2JHH_csv_save_sec', '2JHH_tr_sec_f3', '2JHH_pred_sec_f3', '2JHH_f1_best_iter', '2JHH_f2_best_iter', '2JHHcv_f1', '2JHHcv_f2', '2JHH_meta_fc_est', '2JHH_meta_fc_metafolds', '2JHH_meta_fc_mae_cv_f1', '2JHH_meta_fc_mae_cv_f2', '2JHH_features', '2JHH_meta_fc_mae_cv_f3', '2JHH_f3_best_iter', '2JHHcv_f3', '2JHH_meta_fc_lmae_cv_f1', '2JHH_meta_fc_lmae_cv_f2', '2JHH_meta_fc_lmae_cv_f3', '2JHH_f1-2_metafc_best_iter', '2JHH_f1-3_metafc_best_iter', '2JHH_f2-2_metafc_best_iter', '2JHH_f2-3_metafc_best_iter', '2JHH_f3-2_metafc_best_iter', '2JHH_f3-3_metafc_best_iter']
